In [1]:
import cv2
import numpy as np
import math
import colorsys

In [2]:
#ランダムで座標と値を発生
#本番はリストで受け取るので、この関数は不要
def getRandom2DPoints(width, height, n):
    np.random.seed(202001)
    margin = 10
    entry_pts = np.random.randint(margin, max(height, width), (n*2, 2))
    
    pts = []
    for i, ep in enumerate(entry_pts):
        if ep[0] < margin or (width-margin) < ep[0]:
            continue
        if ep[1] < margin or (height-margin) < ep[1]:
            continue

        # サークル色をHSVで計算
        # 白っぽいを避けるためHSVで計算
        h = i * math.pi * 2.0 / n
        rgb = colorsys.hsv_to_rgb(h, 1.0, 1.0)
        #値aを0-100の間にする
        a = int(rgb[0] * 100)
        pts.append([ep[0],ep[1],a])
    return    pts

In [3]:
#受け取った3値から平均を求める。平均値を参照して色分けする。
def get_color(a1,a2,a3):
    #平均値を求める
    ave = (a1+a2+a3)/3
    
    #値のとりうる範囲を10分割して、白~黒に色分け
    min = 0
    max = 100
    std = (max-min)/10
    
    r=0
    g=0
    b=0
    if(ave<std): #黒
        r=g=b=0
    elif (ave<std*2): 
        r=g=b=25
    elif (ave<std*3): 
        r=g=b=50
    elif (ave<std*4): 
        r=g=b=75
    elif (ave<std*5): 
        r=g=b=100
    elif (ave<std*6): 
        r=g=b=125
    elif (ave<std*7): 
        r=g=b=150
    elif (ave<std*8): 
        r=g=b=175
    elif (ave<std*9): 
        r=g=b=200
    else: #白
        r=g=b=225
        
    return(r,g,b)

In [4]:
def DelaunayDiagram(imgd, subdiv,array):
    height, width = imgd.shape[:2]

    # ドロネーの三角形
    triangles = subdiv.getTriangleList()
    pols = triangles.reshape(-1, 3, 2)
    inner_pts = np.empty((0, 3, 2))
    
    for pa in pols:
        # フレーム外の点は除外
        cnt = 0
        for p in pa:
            if p[0] < 0 or width <= p[0] :
                break
            if p[1] < 0 or height <= p[1] :
                break
            cnt += 1
        # フレーム内の点は有効
        if cnt == 3:
            inner_pts = np.append(inner_pts, [pa], axis=0)
    
    #rgb値を格納する配列を作成
    color_list = np.zeros((len(inner_pts),3))
    
    #3値を渡して色を設定
    for i in range(len(inner_pts)):
        color_list[i]=get_color(array[(int)(inner_pts[i][0][0])][(int)(inner_pts[i][0][1])],array[(int)(inner_pts[i][1][0])][(int)(inner_pts[i][1][1])],array[(int)(inner_pts[i][2][0])][(int)(inner_pts[i][2][1])])
    
    #図形の描画
    for i in range(len(inner_pts)):
        #三角形の描画
        cv2.fillPoly(imgd, [inner_pts[i].astype(int)], color_list[i], 1)
        #三角形の辺の描画
        cv2.line(imgd,(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(0,0,0))

In [5]:
def main():
    height = 240
    width = 320
    
    #点の生成
    pts = getRandom2DPoints(width,height,50)
    
    #白紙を作る
    img = np.full((height, width, 3), 255, np.uint8)
    
    rect = (0, 0, width, height)
    subdiv = cv2.Subdiv2D(rect)

    #(x1,y1,a1)の時、array[x1][y1]=a1とする
    array = np.zeros((width,height))
    
    for p in pts:
        subdiv.insert((int(p[0]), int(p[1])))
        array[p[0]][p[1]] = p[2]
        
    DelaunayDiagram(img, subdiv,array)

    for p in pts:
        color = p[2]
        cv2.circle(img, (p[0],p[1]), 4, color, thickness=-1, lineType=cv2.LINE_AA)
   
    cv2.imwrite('test2.png', img)

In [6]:
if __name__ == '__main__':
    main()